Pole extraction

In [ ]:
import cv2
import numpy as np
import os
import uuid
import matplotlib.pyplot as plt

def extract_and_save_poles(image_path, label_path, free_pole_dir, non_free_pole_dir, display_results=False):
    """
    Processes a single image and its label file, extracting poles from cells
    of class 0 and 2, saves them, and optionally displays the results.
    """
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not read image at {image_path}. Skipping.")
            return

        if not os.path.exists(label_path):
            print(f"Warning: Label file not found for {image_path}. Skipping.")
            return

        img_h, img_w, _ = img.shape

        with open(label_path, 'r') as f:
            lines = f.readlines()

        # Process each labeled cell in the file
        for i, line in enumerate(lines):
            parts = line.strip().split()
            if not parts:
                continue

            class_id = int(parts[0])

            # Only process class 0 (free poles) and class 2 (non-free poles)
            if class_id not in [0, 2]:
                continue

            # Determine the correct output directory based on the class ID
            output_dir = free_pole_dir if class_id == 0 else non_free_pole_dir

            # --- Denormalize Coordinates and Find Rotated Rectangle ---
            coords = np.array([float(p) for p in parts[1:]]).reshape(-1, 2)
            denormalized_points = (coords * np.array([img_w, img_h])).astype(np.float32)
            rect = cv2.minAreaRect(denormalized_points)

            # --- Straighten the Cell and Extract Poles ---
            center, size, angle = rect
            width, height = size

            if width > height:
                width, height = height, width
                angle += 90

            extension = 0 * height
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated_img = cv2.warpAffine(img, M, (img_w, img_h))

            crop_size = (int(width), int(height + 2 * extension))
            super_crop = cv2.getRectSubPix(rotated_img, crop_size, center)

            if super_crop is None or super_crop.size == 0:
                print(f"Warning: Failed to create a valid crop for cell {i} in {image_path}. Skipping.")
                continue

            # --- Slice the Straightened Crop to Get the Poles ---
            mid_point_y = int(extension + height / 2)
            pole1_img = super_crop[0:mid_point_y, :]
            pole2_img = super_crop[mid_point_y:, :]

            # --- Save the Extracted Poles ---
            unique_id = uuid.uuid4().hex[:8]
            base_filename = os.path.splitext(os.path.basename(image_path))[0]
            pole1_filename = f"{base_filename}_cell{i}_pole1_{unique_id}.png"
            pole2_filename = f"{base_filename}_cell{i}_pole2_{unique_id}.png"
            pole1_save_path = os.path.join(output_dir, pole1_filename)
            pole2_save_path = os.path.join(output_dir, pole2_filename)
            cv2.imwrite(pole1_save_path, pole1_img)
            cv2.imwrite(pole2_save_path, pole2_img)

            # --- Display Results if Requested ---
            if display_results:
                print(f"\nDisplaying poles for cell {i} (Class {class_id}) from: {os.path.basename(image_path)}")

                # Create a copy of the original image to draw on
                img_for_display = img.copy()

                # --- Draw the original cell's bounding box for reference ---
                # This box comes directly from the initial detection and should be correct.
                cell_box_points = cv2.boxPoints(rect)
                cv2.drawContours(img_for_display, [cell_box_points.astype(int)], 0, (0, 255, 0), 2) # Green

                # --- Calculate pole bounding boxes on the original image ---
                # This requires transforming the pole crop corners back to the original image space
                M_inv = cv2.invertAffineTransform(M)
                crop_w, crop_h = crop_size

                # Top-left corner of the crop in the rotated image's coordinate system
                top_left_in_rotated = np.array([center[0] - crop_w / 2, center[1] - crop_h / 2])

                # Corners of pole 1 in the rotated image
                p1_corners_rot = np.array([
                    top_left_in_rotated,
                    top_left_in_rotated + [width, 0],
                    top_left_in_rotated + [width, mid_point_y],
                    top_left_in_rotated + [0, mid_point_y]
                ], dtype=np.float32)

                # Corners of pole 2 in the rotated image
                p2_corners_rot = np.array([
                    top_left_in_rotated + [0, mid_point_y],
                    top_left_in_rotated + [width, mid_point_y],
                    top_left_in_rotated + [width, crop_h],
                    top_left_in_rotated + [0, crop_h]
                ], dtype=np.float32)

                # Transform corners back to original image coordinates
                p1_corners_orig = cv2.transform(p1_corners_rot.reshape(-1, 1, 2), M_inv).reshape(-1, 2)
                p2_corners_orig = cv2.transform(p2_corners_rot.reshape(-1, 1, 2), M_inv).reshape(-1, 2)

                # Draw the pole contours on the display image
                cv2.drawContours(img_for_display, [p1_corners_orig.astype(int)], 0, (255, 0, 0), 2) # Blue
                cv2.drawContours(img_for_display, [p2_corners_orig.astype(int)], 0, (0, 0, 255), 2) # Red

                # --- Create and show the plot ---
                fig, axes = plt.subplots(1, 3, figsize=(18, 6))
                axes[0].imshow(cv2.cvtColor(img_for_display, cv2.COLOR_BGR2RGB))
                axes[0].set_title("Original with Pole Boxes")
                axes[0].axis('off')

                axes[1].imshow(cv2.cvtColor(pole1_img, cv2.COLOR_BGR2RGB))
                axes[1].set_title("Extracted Pole 1")
                axes[1].axis('off')

                axes[2].imshow(cv2.cvtColor(pole2_img, cv2.COLOR_BGR2RGB))
                axes[2].set_title("Extracted Pole 2")
                axes[2].axis('off')

                plt.show()

    except Exception as e:
        print(f"An error occurred while processing {image_path}: {e}")


def process_dataset(root_dir, free_pole_dir, non_free_pole_dir, display_results=False):
    """
    Recursively finds all images in the dataset and processes them to extract poles.
    """
    os.makedirs(free_pole_dir, exist_ok=True)
    os.makedirs(non_free_pole_dir, exist_ok=True)
    print(f"Saving free poles to: {free_pole_dir}")
    print(f"Saving non-free poles to: {non_free_pole_dir}")

    images_root = os.path.join(root_dir, 'images')

    for dirpath, _, filenames in os.walk(images_root):
        for filename in filenames:
            if filename.lower().endswith('.png'):
                image_path = os.path.join(dirpath, filename)
                relative_path = os.path.relpath(image_path, images_root)
                label_filename = os.path.splitext(relative_path)[0] + '.txt'
                label_path = os.path.join(root_dir, 'labels', label_filename)

                print(f"Processing: {image_path}")
                extract_and_save_poles(image_path, label_path, free_pole_dir, non_free_pole_dir, display_results)

    print("\nDataset processing complete.")


# --- Main execution ---
if __name__ == '__main__':
    # --- PLEASE CONFIGURE THESE PATHS ---
    DATASET_ROOT_DIRECTORY = "/content/drive/MyDrive/Colab Notebooks/Data/EnhancedPNG"
    OUTPUT_DIRECTORY = "/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced"

    # --- TOGGLE DISPLAY ON/OFF ---
    # Set to True to see the plots for each processed cell.
    # Set to False to run without any pop-up windows (faster).
    SHOW_PLOTS = True

    FREE_POLES_OUTPUT_DIR = os.path.join(OUTPUT_DIRECTORY, "free_poles")
    NON_FREE_POLES_OUTPUT_DIR = os.path.join(OUTPUT_DIRECTORY, "non_free_poles")

    process_dataset(DATASET_ROOT_DIRECTORY, FREE_POLES_OUTPUT_DIR, NON_FREE_POLES_OUTPUT_DIR, display_results=SHOW_PLOTS)


Splitting of Extracted poles dataset

In [ ]:
import os
import shutil
import random

def split_dataset(root_dir, train_ratio=0.9):
    """
    Splits the classification dataset into training and validation sets.

    The expected initial directory structure is:
    root_dir/
    ├── free_poles/
    │   ├── img1.png
    │   └── img2.png
    └── non_free_poles/
        ├── img3.png
        └── img4.png

    The final directory structure will be:
    root_dir/
    ├── train/
    │   ├── free_poles/
    │   └── non_free_poles/
    └── val/
        ├── free_poles/
        └── non_free_poles/

    Args:
        root_dir (str): The path to the directory containing the class folders.
        train_ratio (float): The proportion of the dataset to allocate for training.
    """
    print(f"Starting dataset split in: {root_dir}")
    print(f"Train ratio: {train_ratio}, Validation ratio: {1 - train_ratio}")

    # Define paths for the new train and val directories
    train_dir = os.path.join(root_dir, 'train')
    val_dir = os.path.join(root_dir, 'val')

    # List the class directories (e.g., 'free_poles', 'non_free_poles')
    class_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

    # Filter out 'train' and 'val' if the script is run multiple times
    class_dirs = [d for d in class_dirs if d not in ['train', 'val']]

    if not class_dirs:
        print("Error: No class directories found. The script expects subdirectories named after each class.")
        return

    # Process each class directory
    for class_name in class_dirs:
        print(f"\nProcessing class: {class_name}")

        # Define source and destination paths
        source_class_dir = os.path.join(root_dir, class_name)
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)

        # Create the new train/val subdirectories for the class
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        # Get all image files from the source class directory
        images = [f for f in os.listdir(source_class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        random.shuffle(images) # Shuffle for a random split

        # Calculate the split index
        split_index = int(len(images) * train_ratio)

        # Split the list of images
        train_images = images[:split_index]
        val_images = images[split_index:]

        # Move files to the new directories
        for image in train_images:
            shutil.move(os.path.join(source_class_dir, image), os.path.join(train_class_dir, image))

        for image in val_images:
            shutil.move(os.path.join(source_class_dir, image), os.path.join(val_class_dir, image))

        print(f"Moved {len(train_images)} images to train/{class_name}")
        print(f"Moved {len(val_images)} images to val/{class_name}")

        # After moving all files, the original class folder will be empty and can be removed
        try:
            os.rmdir(source_class_dir)
            print(f"Removed empty source directory: {source_class_dir}")
        except OSError as e:
            print(f"Error removing directory {source_class_dir}: {e}")

    print("\nDataset splitting complete.")


# --- Main execution ---
if __name__ == '__main__':
    # Set this to the directory containing your 'free_poles' and 'non_free_poles' folders
    DATASET_DIRECTORY = "/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced"

    # Call the function to split the dataset
    split_dataset(DATASET_DIRECTORY, train_ratio=0.9)


Writing data.yaml

In [ ]:
import os

def create_data_yaml(root_dir):
    """
    Creates the data.yaml file required for YOLOv8 classification training,
    assuming a train/val split has already been made.

    The expected directory structure is:
    root_dir/
    ├── train/
    │   ├── free_poles/
    │   └── non_free_poles/
    └── val/
        ├── free_poles/
        └── non_free_poles/

    Args:
        root_dir (str): The path to the directory containing train/ and val/ folders.
    """
    # Using an absolute path is generally more robust for YOLO training.
    abs_root_dir = os.path.abspath(root_dir)

    # Define the content for the YAML file.
    # YOLO will look for 'train' and 'val' folders inside the 'path' directory.
    yaml_content = f"""
# The absolute path to the dataset's root directory
path: {abs_root_dir}

# Train/val directories relative to 'path'
train: train
val: val

# Number of classes
nc: 2

# Class names
names:
  0: free_poles
  1: non_free_poles
"""
    # Define the full path for the new file
    yaml_path = os.path.join(abs_root_dir, 'data.yaml')

    # Write the content to the file
    with open(yaml_path, 'w') as f:
        # Using strip() to remove leading/trailing whitespace from the multiline string
        f.write(yaml_content.strip())

    print(f"Successfully created data.yaml at: {yaml_path}")

# --- Main execution ---
if __name__ == '__main__':
    # Set this to the root directory containing your 'train' and 'val' folders
    DATASET_DIRECTORY = "/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced"

    # Call the function to create the file
    create_data_yaml(DATASET_DIRECTORY)

Training Process

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced')

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 classification model (pretrained)
model = YOLO('yolov8x-cls.pt')  # 'n' for nano, choose larger model for better accuracy: yolov8s-cls.pt, yolov8m-cls.pt, yolov8l-cls.pt


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8x-cls.pt') # or yolov8n-cls.pt, etc.

# Train the model
results = model.train(
   data='/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced/',
   epochs=100,
   imgsz=64,
   batch=32,
   workers=4,
   patience=10,
   device=0
)

In [ ]:
from ultralytics import YOLO

# --- 1. Load a powerful, but balanced, model ---
# 'l' (Large) is a great choice for high accuracy without the extreme training time of 'x'.
model = YOLO('yolov8l-cls.pt')

# --- 2. Train the model with the best combination of settings ---
results = model.train(
   # --- Dataset Configuration ---
   data='/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced',

   # --- Core Training Parameters ---
   epochs=100,         # Train for 100 full cycles through the dataset
   imgsz=64,           # Standardize input image size
   batch=32,           # Process 32 images at a time (adjust if you get memory errors)
   patience=20,        # Stop training early if validation accuracy doesn't improve for 20 epochs

   # --- Data Augmentation for Better Generalization ---
   degrees=90,         # Random rotations (-90 to +90 degrees)
   fliplr=0.5,         # 50% chance to flip images horizontally
   flipud=0.5,         # 50% chance to flip images vertically
   scale=0.15,         # Randomly zoom in or out by up to 15%
   hsv_h=0.015,        # Adjust color hue
   hsv_s=0.7,          # Adjust color saturation
   hsv_v=0.4,          # Adjust color brightness/value
   mixup=0.1           # Mix images and labels together to create new training examples
)

# After training, you can find your best model in the 'runs/classify/trainX/weights/best.pt' directory
print("\nTraining complete! Your best model is saved and ready for the inference pipeline.")

Metrics evaluation

In [ ]:
from ultralytics import YOLO

model = YOLO("runs/classify/train2/weights/best.pt")
metrics = model.val(data="/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles")
print(metrics)

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the model and validate (cached results if already validated)
model = YOLO("runs/classify/train2/weights/best.pt")
results = model.val(data="/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles")

# Extract confusion matrix as numpy array
cm = results.confusion_matrix.matrix

# Define class names (replace with your actual class names)
class_names = ["free_poles", "Non_free_poles"]

# If you want integers (counts):
cm_counts = np.rint(cm).astype(int)

# Plot confusion matrix with counts
plt.figure(figsize=(5,4))
sns.heatmap(cm_counts, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Counts)")
plt.show()


Confidence scores

In [ ]:
import os
from ultralytics import YOLO

# -------------------------------------------------------------
# Load Model
# -------------------------------------------------------------
best_pt   = '/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced/runs/classify/train2/weights/best.pt'
cls_model = YOLO(best_pt, task='classify')


# -------------------------------------------------------------
# Predict Helper
# -------------------------------------------------------------
def predict_with_probs(image_path: str):
    """
    Returns: (predicted_class, confidence)
    """
    results = cls_model.predict(
        source=image_path,
        task='classify',
        imgsz=64,
        verbose=False
    )
    probs = results[0].probs.data.tolist()
    top1 = results[0].probs.top1
    return top1, probs[top1]


# -------------------------------------------------------------
# Numerical Evaluation Only
# -------------------------------------------------------------
def evaluate_folder(folder_path: str, true_class: int):
    all_files = sorted([
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))
    ])

    total = len(all_files)
    correct = 0
    wrong = 0

    correct_confs = []
    wrong_confs = []
    all_confs = []

    print(f"\nEvaluating: {folder_path}")
    print(f"True class = {true_class} ({cls_model.names[true_class]})")
    print(f"Total images = {total}")

    for img in all_files:
        pred, prob = predict_with_probs(img)
        all_confs.append(prob)

        if pred == true_class:
            correct += 1
            correct_confs.append(prob)
        else:
            wrong += 1
            wrong_confs.append(prob)

    # -------------------------------
    # Numerical Summary
    # -------------------------------
    print("\n📌 NUMERICAL SUMMARY")
    print("-----------------------")
    print("Total Images        :", total)
    print("Correct Predictions :", correct)
    print("Wrong Predictions   :", wrong)
    print(f"Accuracy            : {correct/total:.4f}")

    if wrong > 0:
        avg_wrong_conf = sum(wrong_confs) / wrong
        print(f"Avg Confidence (Wrong Predictions)  : {avg_wrong_conf:.4f}")
    else:
        print("Avg Confidence (Wrong Predictions)  : N/A")

    if correct > 0:
        avg_correct_conf = sum(correct_confs) / correct
        print(f"Avg Confidence (Correct Predictions): {avg_correct_conf:.4f}")
    else:
        print("Avg Confidence (Correct Predictions): N/A")

    avg_conf_all = sum(all_confs) / total
    print(f"Avg Confidence (All Predictions)    : {avg_conf_all:.4f}")

    print("-----------------------")

    return {
        "total": total,
        "correct": correct,
        "wrong": wrong,
        "avg_conf_wrong": (sum(wrong_confs)/wrong) if wrong>0 else None,
        "avg_conf_correct": (sum(correct_confs)/correct) if correct>0 else None,
        "avg_conf_all": avg_conf_all
    }


# -------------------------------------------------------------
# RUN
# -------------------------------------------------------------
folder = "/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced/train/free_poles"
true_class = 0  # Free pole

evaluate_folder(folder, true_class)

folder = "/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced/train/non_free_poles"
true_class = 1  # Non Free pole

evaluate_folder(folder, true_class)


In [ ]:
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO

# -------------------------------------------------------------
# Load Model
# -------------------------------------------------------------
best_pt   = '/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced/runs/classify/train2/weights/best.pt'
cls_model = YOLO(best_pt, task='classify')


# -------------------------------------------------------------
# Predict Helper
# -------------------------------------------------------------
def predict_with_probs(image_path: str):
    results = cls_model.predict(
        source=image_path,
        task='classify',
        imgsz=64,
        verbose=False
    )
    probs = results[0].probs.data.tolist()
    top1 = results[0].probs.top1
    return top1, probs[top1]


# -------------------------------------------------------------
# Full Evaluation + Plotting
# -------------------------------------------------------------
def evaluate_and_plot(folder_path: str, true_class: int):
    all_files = sorted([
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))
    ])

    total = len(all_files)
    correct = 0
    wrong = 0

    correct_confs = []
    wrong_confs = []

    print(f"\nEvaluating: {folder_path}")
    print(f"True class = {true_class} ({cls_model.names[true_class]})")
    print(f"Total images = {total}")

    # ---------- Predict ----------
    for img in all_files:
        pred, prob = predict_with_probs(img)

        if pred == true_class:
            correct += 1
            correct_confs.append(prob)
        else:
            wrong += 1
            wrong_confs.append(prob)

    # ---------- Summary ----------
    print("\n📌 NUMERICAL SUMMARY")
    print("-----------------------")
    print("Total Images        :", total)
    print("Correct Predictions :", correct)
    print("Wrong Predictions   :", wrong)
    print(f"Accuracy            : {correct/total:.4f}")

    if wrong > 0:
        print(f"Avg Confidence (Wrong Predictions)  : {sum(wrong_confs)/wrong:.4f}")
    else:
        print("Avg Confidence (Wrong Predictions)  : N/A")

    if correct > 0:
        print(f"Avg Confidence (Correct Predictions): {sum(correct_confs)/correct:.4f}")
    else:
        print("Avg Confidence (Correct Predictions): N/A")

    all_confs = correct_confs + wrong_confs
    print(f"Avg Confidence (All Predictions)    : {sum(all_confs)/total:.4f}")
    print("-----------------------")

    # =======================================================
    # 📊 PLOT 1: Correct Predictions Confidence Scores
    # =======================================================
    if len(correct_confs) > 0:
        plt.figure(figsize=(8,4))
        plt.scatter(range(len(correct_confs)), correct_confs, alpha=0.7)
        plt.title("Confidence Scores of Correct Predictions")
        plt.xlabel("Image Index")
        plt.ylabel("Confidence")
        plt.ylim(0, 1.05)
        plt.grid(True, linestyle="--", alpha=0.4)
        plt.show()

    # =======================================================
    # 📊 PLOT 2: Wrong Predictions Confidence Scores
    # =======================================================
    if len(wrong_confs) > 0:
        plt.figure(figsize=(8,4))
        plt.scatter(range(len(wrong_confs)), wrong_confs, color='red', alpha=0.7)
        plt.title("Confidence Scores of Wrong Predictions")
        plt.xlabel("Image Index")
        plt.ylabel("Confidence")
        plt.ylim(0, 1.05)
        plt.grid(True, linestyle="--", alpha=0.4)
        plt.show()



# -------------------------------------------------------------
# RUN
# -------------------------------------------------------------
folder = "/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced/train/free_poles"
true_class = 0  # free pole

evaluate_and_plot(folder, true_class)


folder = "/content/drive/MyDrive/Colab Notebooks/Half_Cell_Poles_enhanced/train/non_free_poles"
true_class = 1 # free pole

evaluate_and_plot(folder, true_class)

Inference on whole image

Ground Truth vs Model Predictions

Analysis of 2-level Process